<a href="https://colab.research.google.com/github/krawch1989/iowa_liquor_sales/blob/stage/main_iowa_liquor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

In [0]:
eth_project_id = 'earnest-star-194115'
client = bigquery.Client(project=eth_project_id)

In [0]:
sql = """
    SELECT name, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10
"""

df = client.query(sql).to_dataframe()

In [14]:
df

,name,count
0,James,5015584
1,John,4885284
2,Robert,4749154
3,Michael,4366524
4,William,3901134
5,Mary,3750838
6,David,3605910
7,Richard,2544867
8,Joseph,2528437
9,Charles,2280600
